In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LOAD THE LIBRARIES

In [ ]:
import keras
import keras_nlp

# LOAD THE GEMMA MODEL

In [ ]:
%%time
llm = keras_nlp.models.GemmaCausalLM.from_preset('gemma_instruct_2b_en')

In [ ]:
dataset_for_qa = pd.read_csv("/kaggle/input/summarized-one-row-per-restaurant/merged_df_generated.csv")
print(dataset_for_qa.shape)
dataset_for_qa.head()

In [ ]:
dataset_for_qa['summarized_review'] = dataset_for_qa['summarized_review'].str.split("Summarized review:").str[-1].str.strip()

In [ ]:
df = dataset_for_qa #.tail(2)

# EXTRACT RESTAURANT HIGHLIGHTS

In [ ]:
# Function to generate dining highlights for each restaurant
def generate_dining_highlights(df):
    dining_highlights = {}
    # Loop through each restaurant and its reviews
    for restaurant_name, group in df.groupby('restaurant_name'):
        reviews = group['summarized_review'].tolist()
        
        # Instruction to generate a highlight summary
        instruction = (
            f"Based on the following reviews for '{restaurant_name}', write 1-2 sentences summarizing the unique aspects and interesting qualities of the dining experience. "
            f"Focus on what makes this restaurant special or worth visiting.\n"
            f"Reviews: {reviews}\n\n"
            f"Return the summary in a brief, engaging format. Convert anything that is not in english language to english while outputting the response."
        )
        
        # Generate the highlight summary with Gemma
        prompt = f"Instruction: {instruction}\nResponse:"
        highlight_summary = llm.generate([prompt], max_length=800)[0]
        
        # Store the highlight for this restaurant
        dining_highlights[restaurant_name] = highlight_summary
    
    return dining_highlights


In [ ]:
%%time
result = generate_dining_highlights(df)

# SAVE THE OUTPUT

In [ ]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['Highlights']).reset_index()
result_df.rename(columns={'index': 'Restaurant Name'}, inplace=True)

In [ ]:
result_df['Highlights'] = result_df ['Highlights'].str.split("Response:").str[-1].str.strip()

In [ ]:
result_df.to_csv("result_df.csv", index = False)